Extrair *dataset*

In [28]:
!unzip tweetSentBR_extracted.zip
%cd tweetSentBR_extracted

Archive:  tweetSentBR_extracted.zip
   creating: tweetSentBR_extracted/
  inflating: tweetSentBR_extracted/testTT.neg  
  inflating: tweetSentBR_extracted/testTT.neu  
  inflating: tweetSentBR_extracted/testTT.pos  
  inflating: tweetSentBR_extracted/trainTT.neg  
  inflating: tweetSentBR_extracted/trainTT.neu  
  inflating: tweetSentBR_extracted/trainTT.pos  
  inflating: tweetSentBR_extracted/tweets.none  
  inflating: tweetSentBR_extracted/tweets.neg  
  inflating: tweetSentBR_extracted/tweets.neu  
  inflating: tweetSentBR_extracted/tweets.pos  
/content/tweetSentBR_extracted


Gerar tabelas separadas por tabulação

In [0]:
import os
import re

corpus = [f for f in os.listdir(os.curdir) if 'TT' in f and not 'tab' in f]

for filepath in corpus:
    with open(filepath) as f:
        text = f.read()

    assert '\t' not in text 
    
    outtext = re.sub(r'(.+?) (.+)',r'\1\t\2', text)
    outpath = re.sub(r'(.+)\.(.+)', r'\1_tab.\2',filepath)
    with open(outpath, 'w') as f:
        f.write(outtext)
        print(outpath)

trainTT_tab.neg
testTT_tab.neu
trainTT_tab.pos
trainTT_tab.neu
testTT_tab.neg
testTT_tab.pos


Formatar arquivo para o formato do mt-dnn

In [0]:
import pandas as pd

splits = {
    'train' : [f for f in os.listdir(os.curdir) if 'train' in f and 'tab' in f],
    'test'  : [f for f in os.listdir(os.curdir) if 'test' in f and 'tab' in f],
}

header = ['id', 'premise']

abbr = {
    'neg': 'Negative', 
    'neu': 'Neutral',
    'pos': 'Positive',
} 

output_header = ['id', 'label', 'premise']

for name, files in splits.items():

    output = pd.DataFrame()
    for f in files:
        table = pd.read_csv(f, index_col = None, sep = '\t', names = header)

        posfix = f.split('.')[1]
        label = abbr[posfix]
        table['label'] = label

        if output.empty:
            output = table
        else:
            output = output.append(table)

    output = output.reindex(columns = output_header)
    output = output.sample(frac = 1)
    
    outpath = 'tweetsent_{}_port.csv'.format(name)
    output.to_csv(outpath, index = None)

    print(outpath)
    print('{} tweets\n'.format(output.shape[0]))

tweetsent_train_port.csv
10980 tweets

tweetsent_test_port.csv
2010 tweets



Substituir pela tradução

In [0]:
import json

files = [f for f in os.listdir(os.curdir) if 'csv' in f and 'port' in f]

with open('tweetsent-dic.json') as f:
    translation = json.load(f)

for filepath in files:
    table = pd.read_csv(filepath)
    table['premise'] = table['premise'].map(translation)
    
    outpath = filepath.replace('_port.csv', '.tsv')
    table.to_csv(outpath, index = None, sep = '\t', header = None)
    
    print(outpath)

tweetsent_test.tsv
tweetsent_train.tsv


Remover marcas html

In [0]:
!pip install ftfy
sh ftfy_tweetsent.sh